# Lab 4 - Matrix Multiplication in Spark using RDDs

Please review Lab 3 before proceeding. 

I've noticed during interactions that some folks are skipping the line below. It is my fault for not explaining it. In Python when you import a file it is never reloaded even if the contents change on disk. If you run the cell below before an import, then it will reload automatically for you.

In [1]:
%load_ext autoreload
%autoreload 2

In [170]:
# make sure your run the cell above before running this
import Lab4_helper

### Matrix Multiplication Reviewed
* Critical to a large number of tasks from graphics and cryptography to graph algorithms and machine learning.
* Computationally intensive. A naive sequential matrix multiplication algorithm has complexity of O(n^3). 
* Algorithms with lower computational complexity exist, but they are not always faster in practice.
* Good candidate for distributed processing
* Every matrix cell is computed using a separate, independent from other cells computation. The computation consumes O(n) input (one matrix row and one matrix column).
* Good candidate for being expressed as a MapReduce computation.
* There are many ways to refresh yourself on matrix muliplication. <a href="https://www.youtube.com/watch?v=kuixY2bCc_0&ab_channel=MathMeeting">Here is one such video.</a>
* <a href="https://en.wikipedia.org/wiki/Matrix_multiplication#Definition">Here is a formal definition</a>

### Why Spark for matrix multiplication? 
If you've ever tried to perform matrix multiplication and you've run out of memory, then you know one of the reasons we might want to use Spark. In general, it is faster to work with a library such as numpy when the matrices are reasonable in size. We would only see the performance benefits of a Spark approach at scale.

### Creating our input

Creating the input for testing purposes is easy. In practice, we would be reading from files or a database. Please review the documentation on <a href="https://spark.apache.org/docs/2.1.1/programming-guide.html#parallelized-collections">parallelized collections</a>.

In order to run the examples in this notebook on the course JupyterHub, you will need to grab the spark context for yourself. This object is already available in databricks, so you do NOT need to run the code below.

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark") \
    .getOrCreate()

sc = spark.sparkContext

Let $A$ be a matrix of size $m \times n$ and $B$ be a matrix of size $n \times s$. Then our goal is to create a matrix $R$ of size $m \times s$. 

Let's start with a concrete example that is represented in what seems like a reasonable way. In general, we use two dimensional arrays to represent lists. Things like:
```python
[[1,2,3],[4,5,6]]
```
We will do that here, but we will write each row as a key,value pair such as:
```python
[('A[1,:]',[1,2,3]),
 ('A[2,:]',[4,5,6])]
```
We'll switch to different formats later for reasons that you will notice while doing this first exercise. If you haven't seen ``A[1,:]`` it means this is the first row and all the columns of the A matrix. Below is how we create the RDDs.

In [152]:
A = [('A[1,:]',[1, 2, 3]),('A[2,:]',[4, 5,6])]
A_RDD = sc.parallelize(A)

B = [('B[:,1]',[7,9,11]),('B[:,2]',[8,10,12])]
B_RDD = sc.parallelize(B)

We can convert these into numpy arrays easily.

In [153]:
import numpy as np
A_mat = np.array(A_RDD.map(lambda v: v[1]).collect())
A_mat

array([[1, 2, 3],
       [4, 5, 6]])

In [154]:
B_mat = np.array(B_RDD.map(lambda v: v[1]).collect())
B_mat

array([[ 7,  9, 11],
       [ 8, 10, 12]])

Let's ask numpy to do our multiplication for us. **Error below is on purpose**. The dot product between two vectors:
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/5bd0b488ad92250b4e7c2f8ac92f700f8aefddd5">
So numpy will calculate the dot product of two vectors each time an entry (circle in image below) is needed:
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Matrix_multiplication_diagram_2.svg/440px-Matrix_multiplication_diagram_2.svg.png">

In [157]:
np.dot(A_mat,B_mat)

ValueError: shapes (2,3) and (2,3) not aligned: 3 (dim 1) != 2 (dim 0)

We have already transposed B in our example to make our map reduce task easier. The ``.dot`` function assumes we have not done the transpose. So in order for numpy to do the multiplication for us, we need to transpose the second matrix (note the .T).

In [158]:
np.dot(A_mat,B_mat.T)

array([[ 58,  64],
       [139, 154]])

Let's pick apart how we got the value 64. This is the dot product of row 1 of A and column 2 of B (Hint: when we discuss matrix we start counting at 1). Our goal then is to compute $n \times s$ values: one for each pair (i, k) of rows from matrix A and columns from matrix B.

To do this we'll need to join the two RDDs together. Why is the following empty?

In [160]:
A_RDD.join(B_RDD).collect()

[]

It's because none of our keys matched. We need to move to a new key, so the join works as expected. Here is what I did:

In [161]:
# These functions needed because we used a string above (don't worry, later I don't do this).
def get_row(s):
    return int(s.split(",")[0].split("[")[-1])

def get_col(s):
    return int(s.split(",")[1].split("]")[0])

In [163]:
A2 = A_RDD.map(lambda kv: ("A x B",[get_row(kv[0]),kv[1]]))
A2.collect()

[('A x B', [1, [1, 2, 3]]), ('A x B', [2, [4, 5, 6]])]

In [164]:
B2 = B_RDD.map(lambda kv: ("A x B",[get_col(kv[0]),kv[1]]))
B2.collect()

[('A x B', [1, [7, 9, 11]]), ('A x B', [2, [8, 10, 12]])]

**Exercise 1:**
Using what I have defined above (A_RDD, B_RDD, A2, B2), the Spark functions (join, map, collect), and the numpy function (np.dot or a loop of your own but why do that...), compute the matrix multiplication of A_RDD and B_RDD.

In [172]:
result = Lab4_helper.exercise_1(A_RDD,B_RDD)
result

[((1, 1), 58), ((1, 2), 64), ((2, 1), 139), ((2, 2), 154)]

In case you want to put it back in the same format

In [174]:
R_mat = np.zeros((2,2))
for row_col,value in result:
    row,col = row_col
    R_mat[row-1,col-1] = value
R_mat

array([[ 58.,  64.],
       [139., 154.]])

Alternative format 1:

'Matrix name', 'row number', 'column number', 'value'

In [141]:
A = [['A',1,1,1],
     ['A',1,2,0],
     ['A',2,1,3],
     ['A',2,2,4],
     ['A',3,1,0],
     ['A',3,2,6],
     ['A',4,1,7],
     ['A',4,2,8]
    ]
A_RDD = sc.parallelize(A)

B = [['B',1,1,7],
     ['B',1,2,8],
     ['B',1,3,9],
     ['B',2,1,0],
     ['B',2,2,11],
     ['B',2,3,0]
    ]
B_RDD = sc.parallelize(B)

In [142]:
A_mat = np.zeros((4,2))
for name,row,col,val in A:
    A_mat[row-1,col-1]=val
A_mat

B_mat = np.zeros((2,3))
for name,row,col,val in B:
    B_mat[row-1,col-1]=val
A_mat,B_mat

(array([[1., 0.],
        [3., 4.],
        [0., 6.],
        [7., 8.]]),
 array([[ 7.,  8.,  9.],
        [ 0., 11.,  0.]]))

In [143]:
np.dot(A_mat,B_mat)

array([[  7.,   8.,   9.],
       [ 21.,  68.,  27.],
       [  0.,  66.,   0.],
       [ 49., 144.,  63.]])

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Matrix_multiplication_diagram_2.svg/440px-Matrix_multiplication_diagram_2.svg.png">

In [176]:
from operator import add
result

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 152.0 failed 1 times, most recent failure: Lost task 3.0 in stage 152.0 (TID 1907, 129.65.17.223, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/tljh/user/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/tljh/user/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/tljh/user/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return

In [145]:
result_mat = np.zeros((4,3))
for row_col,val in result:
    row,col = row_col
    result_mat[row-1,col-1] = val
result_mat

array([[  7.,   8.,   9.],
       [ 21.,  68.,  27.],
       [  0.,  66.,   0.],
       [ 49., 144.,  63.]])

In [146]:
A = [['A',1,1,1],
     ['A',2,1,3],
     ['A',2,2,4],
     ['A',3,2,6],
     ['A',4,1,7],
     ['A',4,2,8]
    ]
A_RDD = sc.parallelize(A)

B = [['B',1,1,7],
     ['B',1,2,8],
     ['B',1,3,9],
     ['B',2,2,11]
    ]
B_RDD = sc.parallelize(B)

In [175]:
from operator import add
result

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 149.0 failed 1 times, most recent failure: Lost task 3.0 in stage 149.0 (TID 1897, 129.65.17.223, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/tljh/user/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/tljh/user/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/tljh/user/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return

In [148]:
result_mat = np.zeros((4,3))
for row_col,val in result:
    row,col = row_col
    result_mat[row-1,col-1] = val
result_mat

array([[  7.,   8.,   9.],
       [ 21.,  68.,  27.],
       [  0.,  66.,   0.],
       [ 49., 144.,  63.]])